In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import csv
import numpy as np
import pandas as pd
import pandas_profiling
import matplotlib.pyplot as plt
from scipy import stats
import pickle
import operator
import glob
from scipy.io.arff import loadarff 
from scipy.stats import ttest_rel, ttest_ind

import seaborn as sns; sns.set_style('white')

from sklearn.utils import resample
from sklearn.metrics import accuracy_score, plot_confusion_matrix, f1_score, plot_roc_curve, roc_auc_score, make_scorer
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import cross_val_score

In [2]:
# 1 = RF, 2 = KNN, 3 = Boost-DT, 4 = Log, 5 = ANN
htru_acc = {
    '1': [0.98, 0.98, 0.9805, 0.9755, 0.9765], 
    '2': [0.9725, 0.97, 0.976, 0.968, 0.9665], 
    '3': [0.968, 0.963, 0.9695, 0.966, 0.966], 
    '4': [0.95, 0.947, 0.9405, 0.9445, 0.946], 
    '5': [0.9555, 0.951, 0.945, 0.9475, 0.951]
}

htru_roc = {
    '1': [0.9961039389672113, 0.9950482758068327, 0.9944005003126954, 0.9939938227892666, 0.9943374337433742], 
    '2': [0.9933840473028117, 0.991702407983629, 0.9901558474046278, 0.9891478729831047, 0.9916826682668267], 
    '3': [0.9907416668000194, 0.9910426452979646, 0.9928655409631019, 0.992091449924185, 0.9924912491249125], 
    '4': [0.9819744043142212, 0.9793541333781851, 0.9739577235772358, 0.975645226464387, 0.980974097409741], 
    '5': [0.9853428893760702, 0.9848344686457544, 0.9826161350844278, 0.9814243591743982, 0.9849294929492949]
}

htru_f1 = {
    '1': [0.980295566502463, 0.9791449426485923, 0.9799897383273473, 0.9750889679715302, 0.9763462506290891], 
    '2': [0.9731051344743277, 0.9692622950819672, 0.9755102040816327, 0.9675126903553299, 0.9663823381836427], 
    '3': [0.968410661401777, 0.9614984391259105, 0.9687339825730394, 0.9655521783181358, 0.9657258064516129], 
    '4': [0.949443882709808, 0.9433155080213904, 0.9367357788410421, 0.9418543740178104, 0.9442148760330578], 
    '5': [0.9552988448016073, 0.9478168264110757, 0.9413020277481323, 0.9455676516329704, 0.9496402877697842]
}

susy_acc = {
    '1': [0.792, 0.771, 0.7865, 0.7855, 0.7955], 
    '2': [0.7555, 0.7505, 0.7485, 0.7575, 0.7585], 
    '3': [0.788, 0.771, 0.779, 0.7945, 0.7915], 
    '4': [0.782, 0.7735, 0.775, 0.7825, 0.791], 
    '5': [0.795, 0.7695, 0.7875, 0.802, 0.7965]
}

susy_roc = {
    '1': [0.8553576715046967, 0.8360234431002238, 0.8509038148843027, 0.8539913823109329, 0.8675452545254526], 
    '2': [0.8160450104815093, 0.8134052965227804, 0.8217005628517823, 0.8281623198146837, 0.8328402840284028], 
    '3': [0.8613070282120626, 0.8457236565469683, 0.8582243902439024, 0.8625510600077615, 0.8665706570657066], 
    '4': [0.8531308508425213, 0.8401872564684776, 0.8512280175109443, 0.8634232309532668, 0.871051105110511], 
    '5': [0.8644909867020851, 0.8489939369911618, 0.8609450906816759, 0.8677230737224496, 0.874019901990199]
}

susy_f1 = {
    '1': [0.7835587929240375, 0.7458379578246392, 0.769811320754717, 0.7684835402050728, 0.783483324510323], 
    '2': [0.7352463454250134, 0.7103888566453859, 0.7007733491969066, 0.7131874630396215, 0.7281935846933033], 
    '3': [0.7832310838445808, 0.7524324324324324, 0.7613390928725702, 0.7791509940891993, 0.7849406910778751],
    '4': [0.7710084033613446, 0.7473508087005019, 0.7483221476510067, 0.7590027700831025, 0.7728260869565217],
    '5': [0.7880041365046535, 0.7517501346257404, 0.7716281569048898, 0.7836065573770491, 0.7859021567596003]
}

higgs_acc = {
    '1': [0.7015, 0.7055, 0.6765, 0.734, 0.735], 
    '2': [0.6115, 0.604, 0.5845, 0.6155, 0.627], 
    '3': [0.7, 0.7015, 0.66, 0.705, 0.705], 
    '4': [0.62, 0.597, 0.5655, 0.5825, 0.5805], 
    '5': [0.652, 0.6425, 0.6115, 0.671, 0.6375]
}

higgs_roc = {
    '1': [0.7690917492118866, 0.7763299374076432, 0.752650406504065, 0.7953613908326033, 0.8038778877887788], 
    '2': [0.6602540765870285, 0.6667057118028441, 0.6420132582864291, 0.6698772959500061, 0.6856835683568356], 
    '3': [0.7646771135043446, 0.7721185690658401, 0.7200940587867417, 0.7734395941604555, 0.7714931493149316], 
    '4': [0.668008193179818, 0.6341340589721718, 0.5881626016260163, 0.6242083448355878, 0.6211941194119412], 
    '5': [0.7151279784288938, 0.6922952933591231, 0.6671164477798623, 0.7281347144040231, 0.6724427442744274]
}

higgs_f1 = {
    '1': [0.6943164362519201, 0.6949766960124287, 0.6656330749354005, 0.7191129883843718, 0.7295918367346939], 
    '2': [0.670063694267516, 0.6299065420560747, 0.6314855875831487, 0.6496583143507972, 0.6728070175438597], 
    '3': [0.702970297029703, 0.7004515805318615, 0.657258064516129, 0.696813977389517, 0.7099311701081613], 
    '4': [0.6367112810707457, 0.6233644859813084, 0.6331785563528916, 0.49849849849849853, 0.6378938282261546], 
    '5': [0.6695156695156694, 0.655421686746988, 0.6850425618159709, 0.6676767676767676, 0.6697038724373576]
}

bit_acc = {
    '1': [0.7155, 0.708, 0.7125, 0.697, 0.6975], 
    '2': [0.644, 0.654, 0.633, 0.622, 0.651], 
    '3': [0.711, 0.708, 0.696, 0.693, 0.657], 
    '4': [0.594, 0.588, 0.566, 0.568, 0.597], 
    '5': [0.623, 0.605, 0.602, 0.65, 0.597]
}

bit_roc = {
    '1': [0.779130694820054, 0.7724879961235188, 0.7850361475922452, 0.7815796896191654, 0.7781128112811282], 
     '2': [0.6924987198156535, 0.7072315596829936, 0.6970331457160724, 0.6805683914047154, 0.6999099909990999], 
     '3': [0.7731883391208333, 0.7814478537188991, 0.7744530331457161, 0.7610191597553122, 0.7076207620762076], 
     '4': [0.5947036373237746, 0.5966592380792196, 0.604416760475297, 0.5862744097843178, 0.6237413741374138], 
     '5': [0.6623263749979996, 0.6489947379170321, 0.6454494058786743, 0.712968241775388, 0.684818481848185]
}

bit_f1 = {
    '1': [0.7247218190614417, 0.7082917082917084, 0.711779448621554, 0.6988071570576541, 0.7015293537247164], 
    '2': [0.6663542642924087, 0.6607843137254902, 0.6623735050597975, 0.6496756255792401, 0.6653883029721955], 
    '3': [0.7068965517241379, 0.6870310825294748, 0.6776246023329798, 0.68989898989899, 0.6727099236641222], 
    '4': [0.6967886482449589, 0.6786271450858035, 0.6640866873065016, 0.6210526315789474, 0.690238278247502], 
    '5': [0.6823925863521483, 0.616504854368932, 0.6721581548599671, 0.6650717703349281, 0.5190930787589499]
}

In [3]:
all_acc = {}
all_acc_1 = []
all_acc_2 = []
all_acc_3 = []
all_acc_4 = []
all_acc_5 = []

all_acc_1.append(sum(htru_acc['1']))
all_acc_1.append(sum(susy_acc['1']))
all_acc_1.append(sum(higgs_acc['1']))    
all_acc_1.append(sum(bit_acc['1']))
all_acc_2.append(sum(htru_acc['2']))
all_acc_2.append(sum(susy_acc['2']))
all_acc_2.append(sum(higgs_acc['2']))    
all_acc_2.append(sum(bit_acc['2']))
all_acc_3.append(sum(htru_acc['3']))
all_acc_3.append(sum(susy_acc['3']))
all_acc_3.append(sum(higgs_acc['3']))    
all_acc_3.append(sum(bit_acc['3']))
all_acc_4.append(sum(htru_acc['4']))
all_acc_4.append(sum(susy_acc['4']))
all_acc_4.append(sum(higgs_acc['4']))    
all_acc_4.append(sum(bit_acc['4']))
all_acc_5.append(sum(htru_acc['5']))
all_acc_5.append(sum(susy_acc['5']))
all_acc_5.append(sum(higgs_acc['5']))    
all_acc_5.append(sum(bit_acc['5']))

all_acc['1']=sum(all_acc_1)/20
all_acc['2']=sum(all_acc_2)/20
all_acc['3']=sum(all_acc_3)/20
all_acc['4']=sum(all_acc_4)/20
all_acc['5']=sum(all_acc_5)/20

print(all_acc)

{'1': 0.7953, '2': 0.7435, '3': 0.7846500000000001, '4': 0.7245250000000001, '5': 0.7495999999999999}


In [4]:
all_roc = {}
all_roc_1 = []
all_roc_2 = []
all_roc_3 = []
all_roc_4 = []
all_roc_5 = []

all_roc_1.append(sum(htru_roc['1']))
all_roc_1.append(sum(susy_roc['1']))
all_roc_1.append(sum(higgs_roc['1']))    
all_roc_1.append(sum(bit_roc['1']))
all_roc_2.append(sum(htru_roc['2']))
all_roc_2.append(sum(susy_roc['2']))
all_roc_2.append(sum(higgs_roc['2']))    
all_roc_2.append(sum(bit_roc['2']))
all_roc_3.append(sum(htru_roc['3']))
all_roc_3.append(sum(susy_roc['3']))
all_roc_3.append(sum(higgs_roc['3']))    
all_roc_3.append(sum(bit_roc['3']))
all_roc_4.append(sum(htru_roc['4']))
all_roc_4.append(sum(susy_roc['4']))
all_roc_4.append(sum(higgs_roc['4']))    
all_roc_4.append(sum(bit_roc['4']))
all_roc_5.append(sum(htru_roc['5']))
all_roc_5.append(sum(susy_roc['5']))
all_roc_5.append(sum(higgs_roc['5']))    
all_roc_5.append(sum(bit_roc['5']))

all_roc['1']=sum(all_roc_1)/20
all_roc['2']=sum(all_roc_2)/20
all_roc['3']=sum(all_roc_3)/20
all_roc['4']=sum(all_roc_4)/20
all_roc['5']=sum(all_roc_5)/20

print(all_roc)

{'1': 0.8515682124563039, '2': 0.7935001018120917, '3': 0.8426580488417933, '4': 0.7656214391927525, '5': 0.8032497377990563}


In [5]:
all_f1 = {}
all_f1_1 = []
all_f1_2 = []
all_f1_3 = []
all_f1_4 = []
all_f1_5 = []


all_f1_1.append(sum(htru_f1['1']))
all_f1_1.append(sum(susy_f1['1']))
all_f1_1.append(sum(higgs_f1['1']))    
all_f1_1.append(sum(bit_f1['1']))
all_f1_2.append(sum(htru_f1['2']))
all_f1_2.append(sum(susy_f1['2']))
all_f1_2.append(sum(higgs_f1['2']))    
all_f1_2.append(sum(bit_f1['2']))
all_f1_3.append(sum(htru_f1['3']))
all_f1_3.append(sum(susy_f1['3']))
all_f1_3.append(sum(higgs_f1['3']))    
all_f1_3.append(sum(bit_f1['3']))
all_f1_4.append(sum(htru_f1['4']))
all_f1_4.append(sum(susy_f1['4']))
all_f1_4.append(sum(higgs_f1['4']))    
all_f1_4.append(sum(bit_f1['4']))
all_f1_5.append(sum(htru_f1['5']))
all_f1_5.append(sum(susy_f1['5']))
all_f1_5.append(sum(higgs_f1['5']))    
all_f1_5.append(sum(bit_f1['5']))

all_f1['1']=sum(all_f1_1)/20
all_f1['2']=sum(all_f1_2)/20
all_f1['3']=sum(all_f1_3)/20
all_f1['4']=sum(all_f1_4)/20
all_f1['5']=sum(all_f1_5)/20

print(all_f1)

{'1': 0.7895400460686851, '2': 0.749902971430383, '3': 0.7796300800956105, '4': 0.744725733848445, '5': 0.756154889170159}


In [6]:
mean_metric_all={}

mean_metric_all['1']=(all_acc['1'] + all_roc['1'] + all_f1['1'])/3
mean_metric_all['2']=(all_acc['2'] + all_roc['2'] + all_f1['2'])/3
mean_metric_all['3']=(all_acc['3'] + all_roc['3'] + all_f1['3'])/3
mean_metric_all['4']=(all_acc['4'] + all_roc['4'] + all_f1['4'])/3
mean_metric_all['5']=(all_acc['5'] + all_roc['5'] + all_f1['5'])/3

print(mean_metric_all)

{'1': 0.8121360861749963, '2': 0.7623010244141583, '3': 0.8023127096458014, '4': 0.7449573910137325, '5': 0.7696682089897383}


In [7]:
# t-tests for means - NH: two sets are statistically indistinguishable, p < 0.05 means reject the null, p > 0.05 retain null (aka *)
for i in range(len(mean_metric_all)):
    print(ttest_rel([all_acc['1'], all_roc['1'], all_f1['1']], [all_acc[str(i + 1)], all_roc[str(i + 1)], all_f1[str(i + 1)]]))

Ttest_relResult(statistic=nan, pvalue=nan)
Ttest_relResult(statistic=9.210753953534981, pvalue=0.011582777803603988)
Ttest_relResult(statistic=19.48656978747843, pvalue=0.002623117698521593)
Ttest_relResult(statistic=5.593889107774433, pvalue=0.030502822365633616)
Ttest_relResult(statistic=9.224451883802589, pvalue=0.011548996639156055)


In [14]:
# t-tests for accs
print(ttest_rel(all_acc_1, all_acc_2))
print(ttest_rel(all_acc_1, all_acc_3))
print(ttest_rel(all_acc_1, all_acc_4))
print(ttest_rel(all_acc_1, all_acc_5))

Ttest_relResult(statistic=2.5322228273312066, pvalue=0.08525791606921863)
Ttest_relResult(statistic=3.2860094328622647, pvalue=0.046216415404471044)
Ttest_relResult(statistic=2.330818102706705, pvalue=0.10207165574406177)
Ttest_relResult(statistic=2.1806163610255695, pvalue=0.11728121875132183)


In [9]:
# t-tests for rocs
print(ttest_rel(all_roc_1, all_roc_2))
print(ttest_rel(all_roc_1, all_roc_3))
print(ttest_rel(all_roc_1, all_roc_4))
print(ttest_rel(all_roc_1, all_roc_5))

Ttest_relResult(statistic=2.308283636461942, pvalue=0.10419456275379478)
Ttest_relResult(statistic=1.40572787759656, pvalue=0.25447193965142945)
Ttest_relResult(statistic=1.8587846005434372, pvalue=0.1600227970397546)
Ttest_relResult(statistic=1.694069135807133, pvalue=0.1888255662612093)


In [10]:
# t-tests for f1s
print(ttest_rel(all_f1_1, all_f1_2))
print(ttest_rel(all_f1_1, all_f1_3))
print(ttest_rel(all_f1_1, all_f1_4))
print(ttest_rel(all_f1_1, all_f1_5))

Ttest_relResult(statistic=3.7226356569347976, pvalue=0.03374571596010911)
Ttest_relResult(statistic=1.9667492834436537, pvalue=0.1439019185469673)
Ttest_relResult(statistic=2.5170293609360184, pvalue=0.08640192348389514)
Ttest_relResult(statistic=1.941028217112582, pvalue=0.14756312449890074)
